In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/2021-ai-w7-p2/sample_submission.csv
/kaggle/input/2021-ai-w7-p2/x_test.csv
/kaggle/input/2021-ai-w7-p2/y_train.csv
/kaggle/input/2021-ai-w7-p2/x_train.csv


In [2]:
#data load

import torch

torch.manual_seed(1)

x_train = pd.DataFrame(pd.read_csv('../input/2021-ai-w7-p2/x_train.csv',engine='python',encoding='CP949'))
y_train  = pd.DataFrame(pd.read_csv('../input/2021-ai-w7-p2/y_train.csv',engine='python',encoding='CP949'))
x_test = pd.DataFrame(pd.read_csv('../input/2021-ai-w7-p2/x_test.csv',engine='python',encoding='CP949'))
submit = pd.DataFrame(pd.read_csv('../input/2021-ai-w7-p2/sample_submission.csv',engine='python',encoding='CP949'))

In [3]:
#gpu 사용

device = 'cuda' if torch.cuda.is_available() else 'cpu'


In [4]:
x_train

,시도명,읍면동명,거주인구,근무인구,방문인구,총 유동인구,평균 속도,평균 소요 시간,평균 기온,일강수량,평균 풍속,월,일,년
0,0,26,76018.965,5009.811,36887.341,117916.117,40.467,21.667,17.750,0.000,2.725,9,25,2019
1,1,24,1232416.968,76043.955,451558.268,1760019.191,38.126,31.632,13.300,14.500,4.500,3,4,2018
2,1,23,134260.946,8412.761,78177.981,220851.688,51.107,45.643,5.250,0.500,4.950,4,7,2018
3,1,25,289327.429,39102.424,287029.303,615459.156,33.067,48.000,26.300,0.500,1.500,8,14,2019
4,1,7,1116617.660,96560.651,524905.301,1738083.612,40.392,26.961,8.400,44.000,4.300,12,1,2019
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21481,0,6,91977.598,14802.819,79885.446,186665.863,41.676,21.676,6.400,0.000,1.600,3,5,2020
21482,0,9,63673.610,3725.663,65813.385,133212.658,48.737,26.105,18.300,7.000,3.700,5,26,2018
21483,1,3,406035.642,46335.550,258079.137,710450.329,49.314,50.943,-6.133,0.833,6.067,1,25,2018
21484,1,25,315737.894,25379.224,283079.548,624196.666,35.312,43.688,8.600,0.000,3.100,3,17,2019


In [5]:
x_test

,시도명,읍면동명,거주인구,근무인구,방문인구,총 유동인구,평균 속도,평균 소요 시간,평균 기온,일강수량,평균 풍속,월,일,년
0,0,28,45737.828,8703.034,91775.025,146215.887,57.286,35.000,6.500,0.0,2.9,1,12,2020
1,1,21,289284.270,48392.749,179649.272,517326.291,50.500,58.912,18.833,0.0,2.3,6,10,2019
2,1,4,72684.218,45632.065,120931.098,239247.380,34.727,60.000,13.200,0.0,1.8,5,10,2019
3,1,12,235752.513,34212.805,186910.679,456875.997,37.167,26.500,11.700,0.0,4.8,3,19,2020
4,1,17,544250.956,64188.293,205182.677,813621.925,31.462,37.769,22.233,0.0,2.3,9,5,2018
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10578,1,37,30949.221,7243.292,73243.733,111436.245,27.500,26.000,0.100,1.0,2.7,2,9,2019
10579,0,39,287544.537,19901.138,230494.219,537939.894,49.130,55.130,20.600,9.0,4.3,7,8,2018
10580,1,35,217597.813,18838.618,150540.187,386976.617,27.500,30.750,0.000,0.0,5.8,2,12,2018
10581,0,6,80350.674,26339.878,101347.505,208038.057,41.000,19.968,18.800,1.0,2.4,10,18,2019


In [6]:
#데이터 정리 - 필요없는 데이터 삭제
x_train = x_train.drop(['시도명','읍면동명','월','일','년'], axis = 1)
x_test = x_test.drop(['시도명','읍면동명','월','일','년'], axis = 1)


In [7]:
#standardscaler를 통한 데이터 정규화
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_test = sc.transform(x_test)


In [8]:
#tensor 자료형으로 변환
x_train = torch.FloatTensor(x_train)
x_test = torch.FloatTensor(x_test)
y_train = torch.FloatTensor(np.array(y_train))

In [9]:
#dataset형태로 변환

from torch.utils.data import DataLoader,TensorDataset

train_set = TensorDataset(x_train,y_train)


In [10]:
x_train.shape

torch.Size([21486, 9])

In [11]:
#학습 파라미터 설정
training_epochs = 500
lr = 0.000001
batch_size = 100
drop_prob = 0.3

In [12]:
#모델 설정
layer1 = torch.nn.Linear(9,512)
layer2 = torch.nn.Linear(512,512)
layer3 = torch.nn.Linear(512,256)
layer4 = torch.nn.Linear(256,128)
layer5 = torch.nn.Linear(128,1)

relu = torch.nn.ReLU()

drop_out = torch.nn.Dropout(p = drop_prob)

model = torch.nn.Sequential(layer1,relu, drop_out,
                           layer2,relu, drop_out,
                           layer3,relu, drop_out,
                           layer4,relu, drop_out,
                           layer5).to(device)


In [13]:
# weight 초기화 (xavier_normal 이용)
torch.nn.init.xavier_normal_(layer1.weight)
torch.nn.init.xavier_normal_(layer2.weight)
torch.nn.init.xavier_normal_(layer3.weight)
torch.nn.init.xavier_normal_(layer4.weight)
torch.nn.init.xavier_normal_(layer5.weight)

Parameter containing:
tensor([[-1.7788e-01, -1.3540e-01,  6.9223e-02,  1.2665e-01, -4.3322e-02,
         -1.6975e-01, -1.6396e-01,  1.1723e-01,  7.1924e-02, -4.9815e-02,
         -5.6659e-02, -7.4052e-02, -2.9620e-01,  6.6823e-03,  6.5631e-02,
         -3.3278e-02,  1.8790e-02,  3.9230e-02, -1.0052e-01,  1.1888e-01,
         -8.9730e-02,  3.1447e-02,  1.0261e-01, -8.2079e-02,  9.5153e-02,
          2.2527e-03, -8.2155e-02,  7.6129e-02,  3.2490e-02, -8.4998e-02,
         -1.0658e-01, -1.6289e-01,  8.1618e-02,  1.3689e-01, -4.1170e-01,
          1.3046e-01,  2.1784e-01,  1.0091e-02, -5.6736e-02, -1.4084e-01,
         -2.2359e-02, -1.8241e-01,  2.0146e-02, -1.9884e-01, -7.0146e-02,
          1.4900e-02,  2.1701e-02, -1.9179e-03, -1.7103e-01,  1.2150e-01,
          1.4483e-01, -3.2716e-03, -9.2192e-02, -7.2767e-02,  1.0724e-02,
         -1.5152e-03,  3.5458e-02,  2.0562e-01, -1.8174e-01, -2.1592e-01,
          1.6197e-01,  1.5477e-02, -9.0184e-02, -3.7653e-02,  4.7564e-02,
         -1.2364

In [14]:
# 손실 함수 , 최적화 함수

loss = torch.nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(),lr = lr)



In [15]:
#dataloader

data_loader = DataLoader(dataset = train_set,batch_size = batch_size , shuffle = True , drop_last = True)

In [16]:
#모델 학습
model.train()

total_batch = len(data_loader)

for e in range(training_epochs):
    avg_cost = 0
    for x, y in data_loader:
        x = x.to(device)
        y = y.to(device)
        
        hypothesis = model(x)
        
        cost = loss(hypothesis, y)
        
        optimizer.zero_grad()
        cost.backward()
        optimizer.step()
        
        avg_cost = cost / total_batch
    print("epoch : ", e + 1 , "cost : ", "{:.9f}".format(cost))
    

epoch :  1 cost :  213695.078125000
epoch :  2 cost :  257506.312500000
epoch :  3 cost :  273026.625000000
epoch :  4 cost :  205572.281250000
epoch :  5 cost :  220539.515625000
epoch :  6 cost :  231933.093750000
epoch :  7 cost :  184781.031250000
epoch :  8 cost :  143249.859375000
epoch :  9 cost :  115410.617187500
epoch :  10 cost :  141685.156250000
epoch :  11 cost :  128990.359375000
epoch :  12 cost :  77445.804687500
epoch :  13 cost :  141745.453125000
epoch :  14 cost :  73988.546875000
epoch :  15 cost :  72397.328125000
epoch :  16 cost :  121525.898437500
epoch :  17 cost :  98768.796875000
epoch :  18 cost :  47272.289062500
epoch :  19 cost :  31021.742187500
epoch :  20 cost :  70695.656250000
epoch :  21 cost :  73546.609375000
epoch :  22 cost :  46145.378906250
epoch :  23 cost :  60195.363281250
epoch :  24 cost :  47294.289062500
epoch :  25 cost :  39905.007812500
epoch :  26 cost :  35744.062500000
epoch :  27 cost :  43681.027343750
epoch :  28 cost :  6549

In [17]:
# 테스트 값 예측

with torch.no_grad():
    model.eval()
    x_test = x_test.to(device)
    
    predict = model(x_test)
predict

tensor([[ 1.1087e+01],
        [ 8.1402e+00],
        [-3.1725e-01],
        ...,
        [ 9.8561e+00],
        [-4.5685e-01],
        [ 7.7387e+02]], device='cuda:0')

In [18]:
submit['predict'] = predict.detach().cpu().numpy().astype(int)

In [19]:
submit.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10583 entries, 0 to 10582
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype
---  ------   --------------  -----
 0   id       10583 non-null  int64
 1   predict  10583 non-null  int64
dtypes: int64(2)
memory usage: 165.5 KB


In [20]:
submit

,id,predict
0,0,11
1,1,8
2,2,0
3,3,1133
4,4,1870
...,...,...
10578,10578,0
10579,10579,5
10580,10580,9
10581,10581,0


In [21]:
submit.to_csv("submission.csv", index = False)